### Question 1 - Create Dataframe


In [1]:
#import libraries
import numpy as np
import pandas as pd
import requests
import json
import csv

#install beautiful soup for web scraping
!pip install watson-developer-cloud==1.5
from bs4 import BeautifulSoup

print("Packages installed")


Packages installed


#### Create the dataframe from Wikipedia

In [2]:
# get url from Wikipedia
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#create a BeautifulSoup object
soup = BeautifulSoup(source, 'lxml')

#create variable for the table of postal codes  
table = soup.find('table')

# Create Pandas Date 
df = pd.DataFrame(columns = ['PostalCode', 'Borough','Neighbourhood'])


# Search all the postcode, borough, neighborhood 
#scan all rows in the first table
for tr in table.find_all('tr'):
    #create empty dataframe
    row_data=[]
    #scan all cells in each row
    for td in tr.find_all('td'):
        #append row to dataframe and strip off non-text info
        row_data.append(td.text.strip())
    #add row from row_data to df when there are 3 columns
    if len(row_data)==3:
        df.loc[len(df)] = row_data


#remove rows with Borough column = 'Not assigned'
df = df[df['Borough'] != 'Not assigned']

df.head(10)


,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


#### Manipulate dataframes to bring boroughs with more than one neighbourhood to the same row. 

In [3]:

#create temporary dataframe and group by postal code
temp_df=df.groupby('PostalCode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))

#reset the index to retain column headings
temp_df=temp_df.reset_index(drop=False)

#Rename the column heading in the original dataframe - will be deleted after merge
df.rename(columns = {'Neighbourhood':'Neighbourhood_old'}, inplace=True)

#merge df and temp_df on the Postal Code column - the common column between the two dfs
df_merge = pd.merge(df, temp_df, on='PostalCode')

#drop the neighbourhood_old column
df_merge.drop(['Neighbourhood_old'],axis=1,inplace=True)

#drop duplicate entries
df_merge.drop_duplicates(inplace=True)

#display first 11 entries
df_merge.head(11)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Etobicoke,Islington Avenue
7,M1B,Scarborough,"Rouge, Malvern"
9,M3B,North York,Don Mills North
10,M4B,East York,"Woodbine Gardens, Parkview Hill"
12,M5B,Downtown Toronto,"Ryerson, Garden District"


#### Get the shape of the notebook

In [4]:
#shape of notebook
df_merge.shape

(103, 3)

### Question 2 - Add Latitude and Longitude

In [7]:
#create dataframe with latitude and longitude by postal code from CSV
df_lat_long = pd.read_csv('http://cocl.us/Geospatial_data')

df_lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
#rename PostalCode to Postal Code in df_merge to match df_lat_long
df_merge.rename(columns = {'PostalCode': 'Postal Code'}, inplace = True)

#merge latitude and longitudes to df_merge
df_loc = pd.merge(df_merge, df_lat_long, on='Postal Code')

df_loc.head(11)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
